<a href="https://colab.research.google.com/github/Bipin6751/Machine-Learning-Collection/blob/master/multiclass_xray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !gdown --id 1V-kAH0GnzHjwG7cfIkOlJSAH2_Q_um5e

In [ ]:
# !unzip chestxray.zip

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.preprocessing import image

In [ ]:
train_path = '/content/chestxray/train'
test_path = '/content/chestxray/test'

In [ ]:
train_datagen = image.ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 0.01,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
)

test_datagen =  image.ImageDataGenerator(
    rescale = 1./255,
)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size = (256, 256),
    batch_size = 32,
    class_mode = 'sparse',
)

In [ ]:
train_generator.class_indices

In [ ]:
test_datagerator = test_datagen.flow_from_directory(
        test_path,
        target_size = (256, 256),
        batch_size = 32,
        class_mode = 'sparse'
)

In [ ]:
# actual = []
# for i in range(test_datagerator.__len__()):
#   for j in test_datagerator[i][1]:
#     actual.append(j)

In [ ]:
# len(actual)

In [ ]:
model = keras.Sequential([
                          layers.Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(256,256,3)),
                          layers.Conv2D(32, (3, 3), activation='relu'),
                          layers.MaxPooling2D(pool_size=(2,2)),

                          layers.Conv2D(64, (3,3), activation='relu'),
                          layers.MaxPooling2D(pool_size=(2,2)),

                          layers.Conv2D(128, (3,3), activation='relu'),
                          layers.MaxPooling2D(pool_size=(2,2)),

                          layers.Conv2D(256, (3,3), activation='relu'),
                          layers.MaxPooling2D(pool_size=(2,2)),
                          layers.Dropout(0.25),

                          layers.Flatten(),
                          layers.Dense(64, activation='relu'),
                          layers.Dropout(0.25),

                          layers.Dense(3, activation='sigmoid'),

])

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=[keras.losses.SparseCategoricalCrossentropy(from_logits=False)],
    metrics=['accuracy'],
)

In [ ]:
mc = keras.callbacks.ModelCheckpoint(filepath="./best_model.h5",
                     monitor="accuracy",
                     verbose=1,
                     save_best_only=True)
es = keras.callbacks.EarlyStopping(monitor="accuracy",
                   min_delta=0.01,
                   patience = 100,
                   verbose=1)
cb = [mc, es]

In [ ]:
his = model.fit(
    train_generator,
    steps_per_epoch = 16,
    epochs = 100,
    callbacks = cb,
)

In [ ]:
model = models.load_model("/content/best_model.h5")

In [ ]:
model.evaluate(train_generator)

In [ ]:
model.evaluate(test_datagerator)

In [ ]:
h = his.history
h.keys()

In [ ]:
plt.plot(h['loss'])
plt.plot(h['accuracy'], 'go--', c='red')
plt.title("Loss vs Accuracy")
plt.show()

In [ ]:
y_predicted = model.predict(test_datagerator)
y_predicted[0]

In [ ]:
y_predicted.shape

In [ ]:
np.argmax(y_predicted[0])

In [ ]:
y_predicted_labels = [np.argmax(i) for i in y_predicted]

In [ ]:
y_predicted_labels[0]

In [ ]:
len(y_predicted_labels)

In [ ]:
test_datagerator.class_indices

In [ ]:
y_actual = []
y_test = []

In [ ]:
import os

In [ ]:
for i in os.listdir('/content/chestxray/test/COVID'):
  img = image.load_img('/content/chestxray/test/COVID/'+ i, target_size=(256, 256))
  img = image.img_to_array(img)/255
  img = np.expand_dims(img, axis = 0)
  p = model.predict(img)
  if np.argmax(p) == 0:
    p = 0
  elif np.argmax(p) == 1:
    p = 1
  else:
    p = 2
  y_test.append(p)
  y_actual.append(0)

In [ ]:
len(y_test)

In [ ]:
for i in os.listdir('/content/chestxray/test/NORMAL'):
  img = image.load_img('/content/chestxray/test/NORMAL/'+ i, target_size=(256, 256))
  img = image.img_to_array(img)/255
  img = np.expand_dims(img, axis = 0)
  p = model.predict(img)
  if np.argmax(p) == 0:
    p = 0
  elif np.argmax(p) == 1:
    p = 1
  else:
    p = 2
  y_test.append(p)
  y_actual.append(1)

In [ ]:
len(y_test)

In [ ]:
for i in os.listdir('/content/chestxray/test/PNEUMONIA'):
  img = image.load_img('/content/chestxray/test/PNEUMONIA/'+ i, target_size=(256, 256))
  img = image.img_to_array(img)/255
  img = np.expand_dims(img, axis = 0)
  p = model.predict(img)
  if np.argmax(p) == 0:
    p = 0
  elif np.argmax(p) == 1:
    p = 1
  else:
    p = 2
  y_test.append(p)
  y_actual.append(2)

In [ ]:
len(y_test)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
cm = confusion_matrix(y_actual, y_test)

In [ ]:
import seaborn as sn
plt.figure(figsize=(10,7))
sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('predicted')
plt.ylabel('Truth')

In [ ]:
print(classification_report(y_actual, y_test))

In [ ]:
def get_img_arr(img_path):
  path = img_path
  img = image.load_img(path, target_size=(256, 256))
  img = image.img_to_array(img)/255
  img = np.expand_dims(img, axis = 0)
  return img

In [ ]:
def make_gracam_heatmap(img_arr, model, last_conv_layer_name, pred_index=None):
  grad_model = tf.keras.models.Model(
      [model.input], [model.get_layer(last_conv_layer_name).output, model.output]
  )

  with tf.GradientTape() as tape:
    last_conv_layer_output, preds = grad_model(img_arr)

    if pred_index is None:
      pred_index = tf.argmax(preds[0])

    class_channel = preds[:, pred_index]

  grads = tape.gradient(class_channel, last_conv_layer_output)

  pooles_grads = tf.reduce_mean(grads, axis=(0,1,2))

  last_conv_layer_output = last_conv_layer_output[0]

  heatmap = last_conv_layer_output @ pooles_grads[..., tf.newaxis]
  heatmap = tf.squeeze(heatmap)
  heatmap = tf.maximum(heatmap, 0)/tf.math.reduce_max(heatmap)
  return heatmap.numpy()

In [ ]:
import matplotlib.cm as cm
from IPython.display import Image, display

In [ ]:
def save_and_display_gradcam(img_path, heatmap, cam_path = "cam.jpg", alpha=0.4):
  img = image.img_to_array(image.load_img(img_path))
  heatmap = np.uint8(255*heatmap)
  jet = cm.get_cmap("jet")
  jet_colors = jet(np.arange(256))[:, :3]
  jet_heatmap = jet_colors[heatmap]

  jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
  jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
  jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

  superimposed_img = jet_heatmap * alpha + img
  superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

  superimposed_img.save(cam_path)

  display(Image(cam_path))

In [ ]:
def image_prediction_and_visualization(path, last_conv_layer_name="conv2d_4", model=model):
  img_array = get_img_arr(path)

  heatmap = make_gracam_heatmap(img_array, model, last_conv_layer_name )

  plt.title("The heat map of the image is")
  plt.imshow(heatmap)
  plt.show()

  pred =  np.argmax(model.predict(img_array))
  print()
  print()
  print()

  if pred == 0:
    print("The patient is COVID infected")
  elif pred == 1:
    print("The patient is NORMAL")
  else:
    print("The patient is PNEUMONIA infected")

  print()
  print()
  print()

  print("Image with heatmap representing region on interest")

  save_and_display_gradcam(path, heatmap)
  print()
  print("The original input image")
  print()

  a = plt.imread(path)
  plt.imshow(a, cmap='gray')
  plt.title("original image")
  plt.show()

In [ ]:
path = "/content/chestxray/test/COVID/COVID-1035.png"
image_prediction_and_visualization(path)

In [ ]:
path = "/content/chestxray/test/NORMAL/IM-0075-0001.jpeg"
image_prediction_and_visualization(path)

In [ ]:
path = "/content/chestxray/test/PNEUMONIA/person1023_virus_1714.jpeg"
image_prediction_and_visualization(path)